In [4]:
# import os
# os.environ['LABDATA_DATBASE_PROJECT']='example' # == 
from labdata.schema import *
import cowpy
import shutup
shutup.please()
import matplotlib.pyplot as plt
%matplotlib qt

In [5]:
Dataset & (Session & (SpikeSorting() & dict(subject_name='JC131')))

subject_name unique mouse id,session_name session identifier,dataset_name,dataset_type,setup_name setup name,notetaker username,note_datetime
JC131,20230805_201448,ephys_g1,ephys,None,None,None
JC131,20230805_201448,orientation,None,None,None,None


In [6]:
from labdata import *
from labdata.schema import *

# list sessions that have sorting results; search by subject_name
sessions = Session & (SpikeSorting() & dict(subject_name = 'JC131')) 

# proj gets just the primary keys
specific_sessions = (Dataset & 
                     sessions &
                     dict(dataset_name = 'orientation')).proj().fetch(as_dict = True) # gets what is exactly in the datajoint

# select the 10th session
selected_session = (Session & specific_sessions[0]).fetch1() 

In [7]:
EphysRecording.ProbeSetting*ProbeConfiguration & selected_session

subject_name unique mouse id,session_name session identifier,dataset_name,probe_num probe number,probe_id probe id to keep track or re-uses,configuration_id,sampling_rate sampling rate,probe_n_channels number of connected channels,probe_gain gain of the probe (multiplication factor),channel_idx index of the channels,channel_shank shank of each channel,"channel_coords channel x,y position"
JC131,20230805_201448,ephys_g1,0,20403312753,3,30000.30000000000000,383,0.762939,=BLOB=,=BLOB=,=BLOB=


In [8]:
# %matplotlib ipympl

In [9]:
# get a dictionary with the configuration of the probe
configuration = (EphysRecording.ProbeSetting*ProbeConfiguration & selected_session).fetch1() 

import pylab as plt

fig = plt.figure(figsize = [3,6]) # figure
fig.add_axes([0.3,0.2,0.7,0.7]) 
# plot the channel locations from the left most shank  
plt.scatter(*configuration['channel_coords'].T, # the x and y position of each channel
            5, # size of the squares
            configuration['channel_shank'], # the shank to color
            marker = 's', # square marker
            edgecolor = 'k',
            lw = 0.5,
            cmap = 'tab20')

plt.ylabel('Distance from tip of shank [$\\mu$m]');
plt.xlabel('Distance from first\nshank edge [$\\mu$m]');

In [10]:
# EphysRecording.ProbeSetting & selected_session
# EphysRecording.ProbeFile & selected_session

# (EphysRecording.ProbeFile & selected_session & 'file_path LIKE "%.ap.meta"')
# File & (EphysRecording.ProbeFile & selected_session & 'file_path LIKE "%.ap.meta"')


(File & (EphysRecording.ProbeFile & selected_session & 'file_path LIKE "%.ap.meta"')).get()

[WindowsPath('C:/Users/danci/data/JC131/20230805_201448/ephys_g1/ephys_g1_imec0/ephys_g1_t0.imec0.ap.meta')]

In [11]:
path = (File & (EphysRecording.ProbeFile & selected_session & 'file_path LIKE "%.ap.meta"')).get()

with open(path[0]) as p:
    print(p.read())

acqApLfSy=384,0,1
appVersion=20230411
fileCreateTime=2023-08-05T20:14:54
fileName=D:/data/JC131/20230805_191054/ephys_g1/ephys_g1_imec0/ephys_g1_t0.imec0.ap.bin
fileSHA1=3392F9869FC9B26312B6F0596EE88000A69947D5
fileSizeBytes=22384103280
fileTimeSecs=968.9982721108232
firstSample=115767272
gateMode=Immediate
imAiRangeMax=0.5
imAiRangeMin=-0.5
imAnyChanFullBand=true
imCalibrated=true
imChan0apGain=80
imDatApi=3.60
imDatBs_fw=2.0.169
imDatBsc_fw=3.2.186
imDatBsc_hw=2.1
imDatBsc_pn=NP2_QBSC_00
imDatBsc_sn=403
imDatFx_hw=1.8
imDatFx_pn=NPM_FLEX_0
imDatFx_sn=0
imDatHs_hw=2.9
imDatHs_pn=NPM_HS_30
imDatHs_sn=20470066
imDatPrb_dock=1
imDatPrb_pn=NP2010
imDatPrb_port=4
imDatPrb_slot=4
imDatPrb_sn=20403312753
imDatPrb_type=24
imIsSvyRun=false
imLEDEnable=false
imMaxInt=8192
imSampRate=30000.325941422594
imStdby=
imSvyMaxBnk=-1
imSvySecPerBnk=35
imTrgRising=true
imTrgSource=0
imroFile=C:/Users/Anne/Desktop/JC131_v1_lp_2_ref.imro
nDataDirs=1
nSavedChans=385
snsApLfSy=384,0,1
snsSaveChanSubset=0:384

In [12]:
# get unit metrics, remove positions that could not be computed and use parameter_set_num to select the sorting.
unitmetrics = pd.DataFrame((UnitMetrics & selected_session & 'position != "NULL"' & 'parameter_set_num = 5').fetch())

fig = plt.figure(figsize = [3,6])
fig.add_axes([0.3,0.2,0.7,0.7]) 

pos = np.vstack(unitmetrics.position.values)
# sort by the inverse of the amplitude so the units are visible
idx = np.argsort(unitmetrics.spike_amplitude.values)[::-1]
plt.scatter(*pos[idx].T,
            unitmetrics.spike_amplitude.values[idx]/10,
            unitmetrics.firing_rate.values[idx], # color by the firing rate
            alpha = 0.8,
            cmap = 'inferno',
            clim = [0,30])
plt.ylabel('Distance from tip of shank [$\\mu$m]');
plt.xlabel('Distance from first\nshank edge [$\\mu$m]');

In [13]:
# gets all spikes, positions and amplitudes; the parameter set is the parameters of the sorter.
units = (SpikeSorting.Unit & selected_session & 'parameter_set_num = 5').fetch(as_dict = True)

In [14]:
pd.DataFrame(units)

,subject_name,session_name,dataset_name,probe_num,parameter_set_num,unit_id,spike_times,spike_positions,spike_amplitudes
0,JC131,20230805_201448,ephys_g1,0,5,0,"[555, 632, 1176, 1257, 1786, 2098, 3104, 3150,...","[[546.06775, 1444.7462], [542.50226, 1448.6548...","[2817.2996, 2968.3464, 2552.5493, 3458.981, 30..."
1,JC131,20230805_201448,ephys_g1,0,5,1,"[413, 3068, 7638, 7941, 7999, 8076, 8181, 9939...","[[557.30536, 1444.1729], [555.8572, 1450.05], ...","[6035.1646, 4055.807, 5659.0195, 4900.1323, 42..."
2,JC131,20230805_201448,ephys_g1,0,5,2,"[3283, 15426, 15742, 17078, 22870, 23454, 2366...","[[539.79987, 1447.5706], [546.7068, 1454.2819]...","[2498.2932, 2306.0881, 2242.637, 1910.4441, 26..."
3,JC131,20230805_201448,ephys_g1,0,5,3,"[1780, 2985, 6719, 7297, 8288, 10472, 10849, 1...","[[551.72833, 1453.3868], [547.4518, 1471.2274]...","[4813.874, 2930.9673, 3412.2424, 3225.7876, 43..."
4,JC131,20230805_201448,ephys_g1,0,5,4,"[49678, 85323, 139064, 192340, 193421, 208520,...","[[555.54395, 1446.2261], [556.0277, 1445.092],...","[8247.518, 14076.351, 7350.754, 9214.625, 7748..."
...,...,...,...,...,...,...,...,...,...
481,JC131,20230805_201448,ephys_g1,0,5,481,"[14302, 16778, 23601, 95799, 206161, 371679, 8...","[[51.40337, 4966.304], [50.630188, 4962.3706],...","[888.1103, 864.3295, 905.7713, 826.35547, 875...."
482,JC131,20230805_201448,ephys_g1,0,5,482,"[3049, 3761, 5153, 14017, 17809, 18386, 18816,...","[[297.61462, 4957.508], [289.81143, 4954.716],...","[731.1472, 849.0779, 742.6442, 750.91473, 841...."
483,JC131,20230805_201448,ephys_g1,0,5,483,"[14441, 14853, 15602, 15746, 15942, 16081, 162...","[[298.82242, 4944.7466], [302.48074, 4949.335]...","[707.05035, 835.17084, 778.3877, 866.03723, 89..."
484,JC131,20230805_201448,ephys_g1,0,5,484,"[1185, 7236, 13374, 15018, 15159, 15281, 15336...","[[42.207863, 4982.059], [52.944412, 5006.39], ...","[750.2998, 732.4088, 672.19415, 686.6945, 736...."


In [15]:
import pylab as plt

In [16]:
spikes = np.hstack([u['spike_times'] for u in units])
amplitudes = np.hstack([u['spike_amplitudes'] for u in units])
positions = np.vstack([u['spike_positions'] for u in units])
# select a random subset of spikes 
idx = np.random.choice(np.arange(len(spikes)),150000)
idx = idx[np.argsort(amplitudes[idx])] # sort by amplitude

fig = plt.figure(figsize=[10,4])
plt.scatter(spikes[idx],positions[idx,1],2,amplitudes[idx],marker = '.',cmap = 'Spectral_r',alpha=0.7,edgecolors=None,clim = [0,20000])
plt.xlim(0,30000*160)
plt.ylabel('Depth ($\\mu$m)')   
plt.xlabel('Time (samples)');

In [17]:
# get the units in a specific shank
units = (SpikeSorting.Unit & (UnitMetrics & 'shank = 2' & 
                              selected_session & 
                              'parameter_set_num = 5')).fetch(as_dict = True) 

spikes = np.hstack([u['spike_times'] for u in units])
amplitudes = np.hstack([u['spike_amplitudes'] for u in units])
positions = np.vstack([u['spike_positions'] for u in units])
# select a random subset of spikes like above
idx = np.random.choice(np.arange(len(spikes)),150000)
idx = idx[np.argsort(amplitudes[idx])] # sort by amplitude

fig = plt.figure(figsize = [3,6])
fig.add_axes([0.3,0.2,0.6,0.7]) 

plt.scatter(positions[idx,0],positions[idx,1],0.5,amplitudes[idx],
            marker = '.',
            cmap = 'Spectral_r',
            alpha=0.5,
            edgecolors=None,
            clim = [0,20000])
plt.axis('off');
plt.ylabel('Distance from tip of shank [$\\mu$m]');
plt.xlabel('Distance from first\nshank edge [$\\mu$m]');

In [18]:
(DatasetEvents.Digital() & selected_session)

subject_name unique mouse id,session_name session identifier,dataset_name,"stream_name which clock is used e.g. btss, nidq, bpod, imecX",event_name,event_timestamps timestamps of the events,event_values event value or count
JC131,20230805_201448,ephys_g1,imec0,6,=BLOB=,=BLOB=
JC131,20230805_201448,ephys_g1,nidq,0,=BLOB=,=BLOB=
JC131,20230805_201448,ephys_g1,nidq,1,=BLOB=,=BLOB=
JC131,20230805_201448,ephys_g1,nidq,2,=BLOB=,=BLOB=
JC131,20230805_201448,ephys_g1,nidq,3,=BLOB=,=BLOB=
JC131,20230805_201448,ephys_g1,nidq,4,=BLOB=,=BLOB=
JC131,20230805_201448,ephys_g1,nidq,5,=BLOB=,=BLOB=
JC131,20230805_201448,ephys_g1,nidq,7,=BLOB=,=BLOB=


In [19]:
plt.figure()
(DatasetEvents.Digital() & selected_session).plot_synced()

In [20]:
StreamSync()

subject_name unique mouse id,session_name session identifier,dataset_name,"stream_name which clock is used e.g. btss, nidq, bpod, imecX",event_name,clock_dataset,"clock_stream which clock is used e.g. btss, nidq, bpod, imecX",clock_stream_event
JC131,20230805_201448,ephys_g1,imec0,6,ephys_g1,nidq,7


In [21]:
# get the spiketimes already synced because of StreamSync
units = (SpikeSorting.Unit & (UnitMetrics  
                              & selected_session 
                              & 'parameter_set_num = 5')).get_spike_times() # run after stream sync

In [22]:
events = pd.DataFrame((DatasetEvents.Digital & selected_session ).fetch_synced()) # get synced dataset events

event_times = events[events['event_name'] == str(1)]['event_timestamps'].iloc[0] # get channel 1
event_values = events[events['event_name'] == str(1)]['event_values'].iloc[0] # because every transition is logged (rise and fall)
event_times = event_times[event_values == 1]

In [23]:
# Read a log file
log = pd.read_csv((File & (Dataset.DataFiles & selected_session 
                   & 'file_path LIKE "%orientation%.csv"')).get()[0])
log['stim_times'] = event_times
stimtimes_sorted = log.sort_values(by='stim')['stim_times'].values

In [24]:
def plot_raster(spks, offset=0.0, height=1.0,
                colors='k',
                ax = None,
                mode = 'scatter',
                lw = 0.5,
                markersize = 2, **kwargs):
    ''' Plot a raster from sets of spiketrains.
            - spks: is a list of spiketrains
            - mode: can be scatter (default) or line. Lines creates a line of height defined by 'height'
        Line mode can be used for exporting to pdf and editing more intuitively
            - height
            - "colors" can be an list of colors (per trial - in line mode)
        Joao Couto - January 2016
    '''
    import pylab as plt
    if ax is None:
        ax = plt.gca()
    nspks = len(spks)
    if mode == 'line': # for exporting to pdf and editing
        if type(colors) is str:
            colors = [colors]*nspks
        for i,(sp,cc) in enumerate(zip(spks,colors)):
            ax.vlines(sp,offset+(i*height),
                    offset+((i+1)*height),
                    colors=cc,**kwargs)
    else: # scatter
        a = np.hstack([i*np.ones_like(s) + offset for i,s in enumerate(spks)])
        b = np.hstack(spks)
        if len(colors) == len(spks):
            colors = np.hstack([[colors[i]]*len(s) for i,s in enumerate(spks) if len(s)]) # to color each trial differently
        s = np.ones_like(b)*markersize
        ax.scatter(b,a,s,c = colors, marker = '|', lw = lw, **kwargs)
        ax.autoscale(tight = True)

In [29]:
# get the spiketimes already synced because of StreamSync
units = (SpikeSorting.Unit & (UnitMetrics  & selected_session & 'parameter_set_num = 5')).get_spike_times()
ts = [u['spike_times'] for u in units]

# get the triggered activity
tpre = 0.5
tpost = 1.5
triggered = []
for sp in ts:
    triggered.append([sp[(sp>(p-tpre)) & (sp<(p+tpost))] - p for p in stimtimes_sorted])

# plot an interactive raster
fig = plt.figure()
from ipywidgets import interact,IntSlider
@interact(iunit = IntSlider(min=0,max=len(triggered)-1))
def pl(iunit):
  fig.clf()
  plot_raster(triggered[iunit],markersize=5)
  plt.hlines(np.arange(0,len(log),log.trial.max()+1),-tpre,tpost,'darkblue',lw = 0.5)
  plt.vlines(0,0,len(log),'r',lw = 0.5)
  plt.xlabel('Time from stim onset (s)')
  plt.ylabel('Trial number')

interactive(children=(IntSlider(value=0, description='iunit', max=485), Output()), _dom_classes=('widget-inter…

In [27]:
DecisionTask().TrialSet().fetch("n_trials").sum()

np.int64(0)

In [28]:
DecisionTask().TrialSet().fetch("n_performed").sum()

np.int64(0)